In [1]:
cd /kaggle/input/transformer-summarizer-ds

/kaggle/input/transformer-summarizer-ds


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import re
import pdb
import time
import utils
import torch
import textwrap
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

wrapper = textwrap.TextWrapper(width=70)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/transformer-summarizer-ds/train.json
/kaggle/input/transformer-summarizer-ds/test.json
/kaggle/input/transformer-summarizer-ds/utils.py
/kaggle/input/transformer-summarizer-ds/val.json


# Import the Dataset

In [3]:
train_data, test_data = utils.get_train_test_data()

# A single example from the dataset
example_summary, example_dialogue = train_data.iloc[10]
print(f"Dialogue: \n{example_dialogue}")
print(f"\nSummary: \n{example_summary}")

Dialogue: 
Lucas: Hey! How was your day?
Demi: Hey there! 
Demi: It was pretty fine, actually, thank you!
Demi: I just got promoted! :D
Lucas: Whoa! Great news!
Lucas: Congratulations!
Lucas: Such a success has to be celebrated.
Demi: I agree! :D
Demi: Tonight at Death & Co.?
Lucas: Sure!
Lucas: See you there at 10pm?
Demi: Yeah! See you there! :D

Summary: 
Demi got promoted. She will celebrate that with Lucas at Death & Co at 10 pm.


# Preprocess the data

In [4]:
document, summary = utils.preprocess(train_data)
document_test, summary_test = utils.preprocess(test_data)

In [5]:
print(document[0])

[SOS] amanda: i baked  cookies. do you want some?  jerry: sure!  amanda: i'll bring you tomorrow :-) [EOS]


## Concatenating summaries and docs to prepare before passing into the tokenizer

In [6]:
docs_and_summary = pd.concat([document, summary], ignore_index=True)

# Punctuation filtering

In [7]:
def apply_filters(text):
    filters = r'[!"#$%&()*+,-./:;<=>?@\\^_`{|}~\t\n]'
    if isinstance(text, str):
        preprocessed_text = re.sub(filters, ' ', text)
        
    else:
        preprocessed_text = [re.sub(filters, ' ', sentence) for sentence in text]
    
    return preprocessed_text

### Applying filtering on training data before tokenizing

In [8]:
filtered_docs_and_summary = apply_filters(docs_and_summary)

# PT

In [9]:
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, trainers

tokenizer = Tokenizer(models.WordLevel(unk_token='[UNK]'))
tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit()
trainer = trainers.WordLevelTrainer(vocab_size=34249, special_tokens=['[UNK]'])

tokenizer.train_from_iterator(filtered_docs_and_summary, trainer)

pytorch_vocab = [token for token in tokenizer.get_vocab().keys()]
print(len(pytorch_vocab))

34249


# TF

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer

filters = '!"#$%&()*+,-./:;<=>?@\\^_`{|}~\t\n'

oov_token = '[UNK]'

tokenizer = Tokenizer(filters=filters, oov_token=oov_token, lower=False)

tokenizer.fit_on_texts(docs_and_summary.tolist())

tf_vocab = [key for key in tokenizer.word_index.keys()]
print(len(tf_vocab))

34249


In [11]:
unique_pytorch_tokens = list(set(pytorch_vocab) - set(tf_vocab))
unique_tf_tokens = list(set(tf_vocab) - set(pytorch_vocab))

print(unique_tf_tokens)
print(unique_pytorch_tokens)

['\u3000', '⊂二二二（\u3000＾ω＾）二⊃', 'comes（\u3000ﾟдﾟ）', '（\u3000´']
['⊂二二二（', 'ﾟдﾟ）', 'comes（', '＾ω＾）二⊃']
